In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 청크 하나가 가질 수 있는 토큰 수
    chunk_overlap=200, # 청크 사이즈의 20% 만큼 중복. 왜 중복필요? => 유사도 검색 시 원하는 답변이 나올 수 있는 확률을 높이기 위함 (문맥 유지)
)

loader = Docx2txtLoader("tax_with_markdown.docx") # 문서 읽기 (tax.docx에서 변경됨)
document_list = loader.load_and_split(text_splitter=text_splitter) # 문서 쪼개기

In [49]:
# 임베딩
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
%pip install --upgrade langchain_pinecone langchain-openai langchain

In [50]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = "tax-index-markdown" # tax-index 대신 tax-index-markdown으로 변경
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_api_key

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name,
    pinecone_api_key=pinecone_api_key,
)


In [82]:
# '연봉 5천만원인 거주자의 소득세는 얼마인가요?' << 내가 원하는 답변이 나오려면 해야하는 질문
# 그러나 거주자 / 직장인 차이를 문서를 보지않는 이상 모른다.  => 'LCEL' 사용하여 질문 변경 필요
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# retrieved_docs = database.similarity_search(query, k=3) # 유사도 검색

In [76]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [77]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: c:\Users\hanjo\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


In [79]:
from langchain_classic import hub # langchain은 deprecated 됨

prompt = hub.pull('rlm/rag-prompt')


In [80]:
from langchain_classic.chains import RetrievalQA

retriever = database.as_retriever()
# retriever 결과 원하는 답변을 받아오지 못함. 왜? 이미지 파싱을 못했기 떄문. 이미지를 markdown으로 변경해야한다.
# (tax_with_markdown.docx 참고. ChatGpt의 경우 이미지를 markdown으로 변경해야 읽을 수 있다.)
retriever.invoke(query)

[Document(id='6a62dd00-925c-4d24-bc73-f29912330d64', metadata={'source': 'tax_with_markdown.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n[제목개정 2014. 1. 1.]\n\n\n\n제4절 세액의 계산 <개정 2009. 12. 31.>\n\n\n\n제1관 세율 <개정 2009. 12. 31.>\n\n\n\n제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,

In [81]:
# LangChain 형태로 질문 답변 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# retrieval 효율화르 위해 딕셔너리도 중요하다.
# 이후 프로젝트에서 참고로 되는 소스를 활용할 때 빈번하게 사용되는 단어는 딕셔너리에 추가해 효율을 높이면 좋다
dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(
    f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전: {dictionary}
    사용자의 질문: {{question}}
    """
)

dictionary_chain = prompt | llm | StrOutputParser()

In [ ]:
tax_chain = {"query": dictionary_chain} | qa_chain # LCEL 사용

In [70]:
ai_response = tax_chain.invoke({"question": query})

In [71]:
ai_response['result']

'연봉 5천만원인 거주자의 소득세는 624만원입니다. 이는 5,000만원 이하 구간의 세율에 따라 결정된 값입니다.'